<a href="https://colab.research.google.com/github/sunjin7725/llama3-8B-passthrough/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate
!pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.8 MB/s eta 0:00:00


In [ ]:
MODEL_NAME = 'SJ-Donald/llama3-passthrough-chat'

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are a chatbot made by kcnet in south korea. you have to answer the question honestly and perfect. If you don`t have any idea, just tell you don`t know!"},
    {"role": "user", "content": "너는 누구니?"},
    {"role": "assistant", "content": "안녕하세요! 저는 KCNET에서 개발한 인공지능 챗봇입니다. 영어와 한국어를 지원하여 다양한 질문에 답해 드리겠습니다. 저는 정보를 최대한 정확하게 제공하려고 노력할 것입니다. 그러나 제가不知道거나 정보가 없는 경우에는 'I don't know'라고 답할 예정이 yourselves."},
    {"role": "user", "content": "대전의 유명한 맛집 소개해줘."}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=4096,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


대전의 맛집 소개해 드리겠습니다! 😊

대전은 한국의 음식 도시로, 다양한 지역 요리를 즐길 수 있는 곳입니다. 다음은 대전의 유명한 맛집 5곳입니다.

1. '한동식' - 대전의 대표적인 전골집입니다. 20년以上의 HISTORY를 자랑하는 곳으로, 다양한 재료를 사용하여 고급한 전골을 제공합니다.
2. '대전식' - 대전의 전통적인 음식 '대전식'을 맛볼 수 있는 곳입니다. 다양한 지역 요리를 즐길 수 있는 곳으로, 특히 '대전식'의 특징적인 음식 '대전식'을 추천합니다.
3. '한산' - 대전의 대표적인 한식집입니다. 다양한 한식 메뉴를 제공하는 곳으로, 특히 '한산'의 특징적인 음식 '한산'의 '한산'을 추천합니다.
4. '대전식' - 대전의 대표적인 '대전식'을 제공하는 곳입니다. 다양한 '대전식' 메뉴를 제공하는 곳으로, 특히 '대전식'의 특징적인 음식 '대전식'을 추천합니다.
5. '한동식' - 대전의 대표적인 '한동식'을 제공하는 곳입니다. 다양한 '한동식' 메뉴를 제공하는 곳으로, 특히 '한동식'의 특징적인 음식 '한동식'을 추천합니다.

이 외에도 다양한 지역 요리를 즐길 수 있는 곳이 많습니다. 대전의 음식 문화를 즐기고 싶다면, '대전식'의 다양한 음식을 추천합니다! 😊
